<a href="https://colab.research.google.com/github/kategimranova/Cinderella_Telegram/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import time
import pandas as pd
import numpy as np

In [2]:
train_solution = pd.read_csv("train_solution.csv")
train_data = pd.read_csv("train_data.csv")

In [3]:
train = train_data.merge(train_solution, on='id')

In [4]:
train.head()

,id,message,category
0,271828,Over $616 million in Bitcoin was electrocated ...,1
1,271829,Quiz: Thursday or friday?,0
2,271830,The Australian Revenue Authority will start co...,1
3,271831,Let's continue😉. I present to you my new review,2
4,271832,Here comes your future palette.,2


In [5]:
texts = list(train['message']) #список предложений

In [7]:
#оценка сбалансированности классов
print(train[train.category == 0].shape[0])
print(train[train.category == 1].shape[0])
print(train[train.category == 2].shape[0])

1428
1199
1217


In [10]:
import re

#Функции для очистки текстов. переводит все в нижний регистр, оставляет только слова, числа меняет на решетки
# в зависимости от количеcтва цифр и Меняет определенные сокращения (contraction_dict)

def clean_text(x):
    x = x.lower()
    x = replace_contractions(x)
    pattern = r'[^a-zA-z0-9\s]'
    x = re.sub(pattern, '', x)
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [11]:
train['cleaned'] = train['message'].apply(lambda x: clean_text(x))

In [12]:
words = []
for text in train['cleaned']:
  words += text.split()

In [27]:
max_features = len(set(words)) #количество уникальных слов
max_features

16958

In [13]:
train.head()

,id,message,category,cleaned
0,271828,Over $616 million in Bitcoin was electrocated ...,1,over ### million in bitcoin was electrocated i...
1,271829,Quiz: Thursday or friday?,0,quiz thursday or friday
2,271830,The Australian Revenue Authority will start co...,1,the australian revenue authority will start co...
3,271831,Let's continue😉. I present to you my new review,2,let us continue i present to you my new review
4,271832,Here comes your future palette.,2,here comes your future palette


In [17]:
#Разделение на трейн и тест (валидацию) с учетом балансов класса (stratify) и фиксированным рэндом стейтом
train_X, test_X, train_y, test_y = train_test_split(train['cleaned'], train['category'],
                                                    stratify=train['category'], 
                                                    test_size=0.25, random_state=42)

In [18]:
print("Train shape : ",train_X.shape)
print("Test shape : ",test_X.shape)

Train shape :  (2883,)
Test shape :  (961,)


In [20]:
train['len'] = train['cleaned'].apply(lambda x: len(x))

In [22]:
#Определяем максимальную длину текстов
maxlen  = train['len'].max()

In [28]:
#Токенизация предложений по словам 
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [29]:
#добавление паддингов (чтобы все тексты были одинаковой длины)
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [32]:
le = LabelEncoder()
train_y = le.fit_transform(train_y.values)
test_y = le.transform(test_y.values)

In [37]:
embed_size = 300 #размер эмбеддингов

In [36]:
class CNN_Text(nn.Module):
    def __init__(self):
        super(CNN_Text, self).__init__()
        filter_sizes = [1,2,3,5]
        num_filters = 36
        n_classes = 3
        self.embedding = nn.Embedding(max_features, embed_size)
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.15)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, n_classes)


    def forward(self, x):
        x = self.embedding(x)  
        x = x.unsqueeze(1)  
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = torch.cat(x, 1)
        x = self.dropout(x)  
        logit = self.fc1(x) 
        return logit

In [40]:
n_epochs = 13
batch_size = 512
n_splits = 5
model = CNN_Text()
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
model.cuda()
x_train = torch.tensor(train_X, dtype=torch.long).cuda()
y_train = torch.tensor(train_y, dtype=torch.long).cuda()
x_cv = torch.tensor(test_X, dtype=torch.long).cuda()
y_cv = torch.tensor(test_y, dtype=torch.long).cuda()
train = torch.utils.data.TensorDataset(x_train, y_train)
valid = torch.utils.data.TensorDataset(x_cv, y_cv)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

train_loss = []
valid_loss = []

for epoch in range(n_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.  
    for i, (x_batch, y_batch) in enumerate(train_loader):
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)
    
    model.eval()        
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv),len(le.classes_)))
    
    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        val_preds[i * batch_size:(i+1) * batch_size] =F.softmax(y_pred).cpu().numpy()
    
    val_accuracy = sum(val_preds.argmax(axis=1)==test_y)/len(test_y)
    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)
    elapsed_time = time.time() - start_time 
    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f}  \t val_acc={:.4f}  \t time={:.2f}s'.format(
                epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/13 	 loss=457.3944 	 val_loss=321.1057  	 val_acc=0.7534  	 time=10.56s
Epoch 2/13 	 loss=314.3801 	 val_loss=275.7043  	 val_acc=0.7836  	 time=10.49s
Epoch 3/13 	 loss=262.1659 	 val_loss=248.3005  	 val_acc=0.8002  	 time=10.57s
Epoch 4/13 	 loss=218.0238 	 val_loss=227.6057  	 val_acc=0.8314  	 time=10.63s
Epoch 5/13 	 loss=188.5969 	 val_loss=216.1898  	 val_acc=0.8273  	 time=10.64s
Epoch 6/13 	 loss=169.2611 	 val_loss=206.5979  	 val_acc=0.8439  	 time=10.68s
Epoch 7/13 	 loss=144.8870 	 val_loss=199.8827  	 val_acc=0.8491  	 time=10.71s
Epoch 8/13 	 loss=128.1963 	 val_loss=193.8829  	 val_acc=0.8564  	 time=10.83s
Epoch 9/13 	 loss=111.2685 	 val_loss=190.0008  	 val_acc=0.8606  	 time=10.76s
Epoch 10/13 	 loss=98.8440 	 val_loss=185.4957  	 val_acc=0.8595  	 time=10.80s
Epoch 11/13 	 loss=86.8472 	 val_loss=183.7496  	 val_acc=0.8626  	 time=10.85s
Epoch 12/13 	 loss=77.4107 	 val_loss=181.6650  	 val_acc=0.8647  	 time=10.88s
Epoch 13/13 	 loss=67.1676 	 val_loss=17

In [ ]:
torch.save(model.state_dict(), "model_new1.pt")